In [ ]:
from keras.preprocessing.sequence import pad_sequences


ImportError: ignored

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [11]:
#/content/drive/MyDrive/NLP/stuff/datasets/partitive_group_nombank.clean.test
%cd  /content/drive/MyDrive/NLP/stuff/datasets

/content/drive/MyDrive/NLP/stuff/datasets


partitive_group_nombank.clean.dev   partitive_group_nombank.clean.train
partitive_group_nombank.clean.test


## make features

# New section

In [ ]:
from nltk.stem import PorterStemmer
import csv
# PERCENT TASK


ps = PorterStemmer()

data = list()


if __name__ == '__main__':

	train_file = open('datasets/partitive_group_nombank.clean.train')
	train_file_lines = train_file.readlines()
	train_file.close()
	
	# reading the training file

	count_Sentences = 0  

	outfile = open('part_dataset.train', 'w+')
	i = 0 

	# outfile.write(f'word\tpos\tbio\tstemmed\tpos_back1\tword_back1\tstemmed_back1\tbio_back1\tpos_back2\tword_back2\tstemmed_back2\tbio_back2\tpos_plus1\tword_plus1\tstemmed_plus1\tbio_plus1\tpos_plus2\tword_plus2\tstemmed_plus2\tbio_plus2\tpred_column\n')
	while i < len(train_file_lines):
		
		line = train_file_lines[i]
		print('current line:', line)
		prev_line, prev_prev_line = '', ''
		if i > 0:
			prev_line = train_file_lines[i-1]
		if i > 1:
			prev_prev_line = train_file_lines[i-2]
		next_line, next_next_line = '', ''
		if i < len(train_file_lines) - 1:
			next_line = train_file_lines[i+1]
		if i < len(train_file_lines) - 2:
			next_next_line = train_file_lines[i+2]

		# print('prev:', prev_line)
		# print('prev2:', prev_prev_line)
		# print('next:', next_line)
		# print('next2:', next_next_line)
		


		# if i == 5:
		# 	exit()

		write_line = ''
		if len(line.strip()) == 0:
			write_line = ''
			outfile.write('\n')
		else:
			line_parts = line.split()
			word = line_parts[0]
			pos = line_parts[1]
			bio = line_parts[2]
			word_id = line_parts[3]
			sent_id = line_parts[4]

			is_pred = False
			part_type = None
			is_arg = False
			is_support = False

			if len(line_parts) > 5:
				role = line_parts[5]
				if role == 'PRED':
					is_pred = True
					part_type = line_parts[6]
				elif role == 'ARG1':
					is_arg = True
				elif role == 'SUPPORT':
					is_support = True
			

			stemmed_word = ps.stem(word)

			write_line = f'{word}\tPOS={pos}\tBIO={bio}\tSTEMMED_WORD={stemmed_word}\t'

			# ---------------------- previous lines ----------------------
			if len(prev_line.strip()) == 0:
				write_line += f'pos_back_1=START\tword_back_1=START\tstemmed_back_1=START\tbio_back_1=O\t'
			else:
				prev_parts = prev_line.split()
				write_line += f'pos_back_1={prev_parts[1]}\tword_back_1={prev_parts[0]}\tstemmed_back_1={ps.stem(prev_parts[0])}\tbio_back_1={prev_parts[2]}\t'

			if len(prev_prev_line.strip()) == 0:
				write_line += f'pos_back_2=START\tword_back_2=START\tstemmed_back_2=START\tbio_back_2=O\t'
			else:
				prev2_parts = prev_prev_line.split()
				write_line += f'pos_back_2={prev2_parts[1]}\tword_back_2={prev2_parts[0]}\tstemmed_back_2={ps.stem(prev2_parts[0])}\tbio_back_2={prev2_parts[2]}\t'

			
			# ----------------------- next lines ------------------------
			if len(next_line.strip()) == 0:
				write_line += f'pos_plus_1=END\tword_plus_1=END\tstemmed_plus_1=END\tbio_plus_1=O\t'
			else:
				next_parts = next_line.split()
				write_line += f'pos_plus_1={next_parts[1]}\tword_plus_1={next_parts[0]}\tstemmed_plus_1={ps.stem(next_parts[0])}\tbio_plus_1={next_parts[2]}\t'

			# print('length of next next line:', len(next_next_line), 'next next line:', next_next_line,'done')
			if len(next_next_line.strip()) == 0:
				write_line += f'pos_plus_2=END\tword_plus_2=END\tstemmed_plus_2=END\tbio_plus_2=O\t'
			else:
				next_next_parts = next_next_line.split() 
				# print('next next parts:', next_next_parts)
				write_line += f'pos_plus_2={next_next_parts[1]}\tword_plus_2={next_next_parts[0]}\tstemmed_plus_2={ps.stem(next_next_parts[0])}\tbio_plus_2={next_next_parts[2]}\t'

			write_line += f'token_count={word_id}\tsentence_id={sent_id}\tis_support={is_support}\tis_pred={is_pred}\tpart_type={part_type}\t{is_arg}\n'

			outfile.write(write_line)




		i += 1
			
			
		

		




	# # write data to csv
	# with open('train_features.csv', 'w') as out:
	# 	csv_out = csv.writer(out)
	# 	csv_out.writerows(data)




FileNotFoundError: ignored

# New section

In [ ]:
import sklearn
import nltk
from nltk.classify import MaxentClassifier
from nltk.classify.util import accuracy
import collections
from nltk import metrics
from nltk.classify import NaiveBayesClassifier

In [ ]:
def convert_to_featureset(lines):
	featureset = list()
	for i in range(len(lines)):
		line = lines[i]
		if len(line.strip()) != 0:
			
			parts = line.split()
			label = parts[-1]
			features_list = parts[:len(parts) - 1]
			# print('features list:', features_list)
			features_dict = dict()
			for j in range(len(features_list)):
				feature = features_list[j]
				# print(feature)
				key, value = None, None
				if j == 0: # its the word
					key = 'word'
					value = feature
				else:
					key, value = feature.split('=')
					if  key == 'token_count' or key == 'sentence_id' or key == 'is_pred' or key == 'part_type':
						continue
						
				
				features_dict[key] = value

			pair = (features_dict, label)
			featureset.append(pair)

	return featureset

In [ ]:
def get_all_details(lines):
	featureset = list()
	for i in range(len(lines)):
		line = lines[i]
		if len(line.strip()) != 0:
			
			parts = line.split()
			label = parts[-1]
			features_list = parts[:len(parts) - 1]
			# print('features list:', features_list)
			features_dict = dict()
			for j in range(len(features_list)):
				feature = features_list[j]
				# print(feature)
				key, value = None, None
				if j == 0: # its the word
					key = 'word'
					value = feature
				else:
					key, value = feature.split('=')
				
				features_dict[key] = value

			pair = (features_dict, label)
			featureset.append(pair)

	return featureset

In [ ]:
train_data = open('part_dataset.train')
train_data_lines = train_data.readlines()
train_data.close()
train_encoding = convert_to_featureset(train_data_lines)
train_encoding[0]

({'word': 'But',
  'POS': 'CC',
  'BIO': 'O',
  'STEMMED_WORD': 'but',
  'pos_back_1': 'START',
  'word_back_1': 'START',
  'stemmed_back_1': 'START',
  'bio_back_1': 'O',
  'pos_back_2': 'START',
  'word_back_2': 'START',
  'stemmed_back_2': 'START',
  'bio_back_2': 'O',
  'pos_plus_1': 'IN',
  'word_plus_1': 'about',
  'stemmed_plus_1': 'about',
  'bio_plus_1': 'B-NP',
  'pos_plus_2': 'CD',
  'word_plus_2': '25',
  'stemmed_plus_2': '25',
  'bio_plus_2': 'I-NP',
  'is_support': 'False'},
 'False')

In [ ]:
train_encoding[6]

({'word': 'insiders',
  'POS': 'NNS',
  'BIO': 'I-NP',
  'STEMMED_WORD': 'insid',
  'pos_back_1': 'DT',
  'word_back_1': 'the',
  'stemmed_back_1': 'the',
  'bio_back_1': 'B-NP',
  'pos_back_2': 'IN',
  'word_back_2': 'of',
  'stemmed_back_2': 'of',
  'bio_back_2': 'B-PP',
  'pos_plus_1': 'COMMA',
  'word_plus_1': 'COMMA',
  'stemmed_plus_1': 'comma',
  'bio_plus_1': 'O',
  'pos_plus_2': 'VBG',
  'word_plus_2': 'according',
  'stemmed_plus_2': 'accord',
  'bio_plus_2': 'B-PP',
  'is_support': 'False'},
 'True')

In [ ]:
nb_classifier = NaiveBayesClassifier.train(train_encoding)

## Dev set

In [ ]:
dev_data = open('part_dataset.dev')
dev_data_lines = dev_data.readlines()
dev_data.close()
dev_encoding= convert_to_featureset(dev_data_lines)
dev_all_cols = get_all_details(dev_data_lines)

In [ ]:
dev_encoding[0]

({'word': 'The',
  'POS': 'DT',
  'BIO': 'B-NP',
  'STEMMED_WORD': 'the',
  'pos_back_1': 'START',
  'word_back_1': 'START',
  'stemmed_back_1': 'START',
  'bio_back_1': 'O',
  'pos_back_2': 'START',
  'word_back_2': 'START',
  'stemmed_back_2': 'START',
  'bio_back_2': 'O',
  'pos_plus_1': 'NN',
  'word_plus_1': 'consensus',
  'stemmed_plus_1': 'consensu',
  'bio_plus_1': 'I-NP',
  'pos_plus_2': 'NN',
  'word_plus_2': 'view',
  'stemmed_plus_2': 'view',
  'bio_plus_2': 'I-NP',
  'is_support': 'False'},
 'False')

In [ ]:
print(dev_all_cols[0])

({'word': 'The', 'POS': 'DT', 'BIO': 'B-NP', 'STEMMED_WORD': 'the', 'pos_back_1': 'START', 'word_back_1': 'START', 'stemmed_back_1': 'START', 'bio_back_1': 'O', 'pos_back_2': 'START', 'word_back_2': 'START', 'stemmed_back_2': 'START', 'bio_back_2': 'O', 'pos_plus_1': 'NN', 'word_plus_1': 'consensus', 'stemmed_plus_1': 'consensu', 'bio_plus_1': 'I-NP', 'pos_plus_2': 'NN', 'word_plus_2': 'view', 'stemmed_plus_2': 'view', 'bio_plus_2': 'I-NP', 'token_count': '0', 'sentence_id': '0', 'is_support': 'False', 'is_pred': 'False', 'part_type': 'None'}, 'False')


In [ ]:
sentence_features = list()
sentence_count = 0
for e in dev_encoding:
	sentence_features.append(e[0])

len(sentence_features)

11992

In [ ]:
results = nb_classifier.classify_many(sentence_features)

In [ ]:
# write to output file
out_file = open('part_nopred.txt', 'w+')
current_sentence_count = 0 

for i in range(len(results)):
	out_line = ''
	# print('sentence count check -----------------')
	if int(dev_all_cols[i][0]['sentence_id']) == current_sentence_count + 1:
		current_sentence_count = int(dev_all_cols[i][0]['sentence_id'])
		out_file.write('\n')
	word = dev_all_cols[i][0]['word']
	pos = dev_all_cols[i][0]['POS']
	bio = dev_all_cols[i][0]['BIO']
	token_count = dev_all_cols[i][0]['token_count']
	sentence = dev_all_cols[i][0]['sentence_id']
	out_line += f'{word}\t{pos}\t{bio}\t{token_count}\t{sentence}\t'
	if dev_all_cols[i][0]['is_support'] == 'True':
		out_line += f'SUPPORT\t'
	elif dev_all_cols[i][0]['is_pred'] == 'True':
		part_type = dev_all_cols[i][0]['part_type']
		out_line += f'PRED\t{part_type}\t'
	elif results[i] == 'True':
		out_line += f'ARG1\t'
	out_file.write(out_line+'\n')
out_file.close()

In [ ]:
from score import score_file_with_NNP_adjustment

score_file_with_NNP_adjustment('part_nopred.txt','Partitive-Files/partitive_group_nombank.clean.dev', 'ARG1')

System [10, 11, 30, 72, 93, 114, 138, 180, 194, 215, 228, 248, 267, 292, 315, 345, 391, 406, 436, 479, 508, 509, 510, 511, 526, 569, 583, 634, 659, 660, 707, 727, 749, 775, 818, 843, 867, 889, 917, 983, 1022, 1025, 1038, 1059, 1093, 1105, 1148, 1172, 1174, 1213, 1236, 1283, 1325, 1340, 1361, 1387, 1413, 1414, 1415, 1417, 1418, 1438, 1439, 1440, 1442, 1443, 1509, 1510, 1523, 1553, 1572, 1586, 1620, 1662, 1663, 1724, 1741, 1756, 1764, 1787, 1806, 1830, 1869, 1877, 1904, 1911, 1936, 1959, 1975, 2009, 2046, 2070, 2111, 2143, 2179, 2199, 2230, 2246, 2267, 2294, 2322, 2379, 2380, 2381, 2392, 2415, 2470, 2495, 2513, 2538, 2558, 2625, 2673, 2683, 2700, 2709, 2724, 2748, 2784, 2785, 2786, 2787, 2834, 2868, 2892, 2924, 2956, 2976, 2977, 2978, 3016, 3042, 3099, 3150, 3243, 3277, 3305, 3324, 3347, 3385, 3403, 3426, 3481, 3493, 3514, 3540, 3553, 3579, 3580, 3581, 3615, 3623, 3646, 3678, 3700, 3746, 3797, 3808, 4007, 4008, 4055, 4196, 4250, 4286, 4319, 4354, 4395, 4435, 4463, 4547, 4557, 4661, 4678,

## TEST SET

In [ ]:
test_data = open('part_dataset.test')
test_data_lines = test_data.readlines()
test_data.close()
test_encoding= convert_to_featureset(test_data_lines)
test_all_cols = get_all_details(test_data_lines)

In [ ]:
test_all_cols[1]

({'word': 'in',
  'POS': 'IN',
  'BIO': 'B-PP',
  'STEMMED_WORD': 'in',
  'pos_back_1': 'RB',
  'word_back_1': 'Then',
  'stemmed_back_1': 'then',
  'bio_back_1': 'B-ADVP',
  'pos_back_2': 'START',
  'word_back_2': 'START',
  'stemmed_back_2': 'START',
  'bio_back_2': 'O',
  'pos_plus_1': 'DT',
  'word_plus_1': 'a',
  'stemmed_plus_1': 'a',
  'bio_plus_1': 'B-NP',
  'pos_plus_2': 'NN',
  'word_plus_2': 'lightning',
  'stemmed_plus_2': 'lightn',
  'bio_plus_2': 'I-NP',
  'token_count': '1',
  'sentence_id': '0',
  'is_support': 'False',
  'is_pred': 'False',
  'part_type': 'None'},
 'False')

In [ ]:
test_features = list()
sentence_count = 0
for e in test_encoding:
	test_features.append(e[0])

len(test_features)

18982

In [ ]:
test_results = nb_classifier.classify_many(test_features)

In [ ]:
# write to output file
out_file = open('FINAL_PART.txt', 'w+')
current_sentence_count = 0 

for i in range(len(test_results)):
	out_line = ''
	# print('sentence count check -----------------')
	if int(test_all_cols[i][0]['sentence_id']) == current_sentence_count + 1:
		current_sentence_count = int(test_all_cols[i][0]['sentence_id'])
		out_file.write('\n')
	word = test_all_cols[i][0]['word']
	pos = test_all_cols[i][0]['POS']
	bio = test_all_cols[i][0]['BIO']
	token_count = test_all_cols[i][0]['token_count']
	sentence = test_all_cols[i][0]['sentence_id']
	out_line += f'{word}\t{pos}\t{bio}\t{token_count}\t{sentence}\t'
	if test_all_cols[i][0]['is_support'] == 'True':
		out_line += f'SUPPORT\t'
	elif test_all_cols[i][0]['is_pred'] == 'True':
		part_type = test_all_cols[i][0]['part_type']
		out_line += f'PRED\t{part_type}\t'
	elif test_results[i] == 'True':
		out_line += f'ARG1\t'
	out_file.write(out_line.strip())
	out_file.write('\n')
out_file.close()

In [ ]:
score_file_with_NNP_adjustment('FINAL_PART.txt','Partitive-Files/partitive_group_nombank.clean.test', 'ARG1')

System [9, 62, 78, 103, 116, 154, 169, 195, 229, 257, 288, 320, 355, 380, 421, 438, 462, 501, 523, 557, 581, 612, 637, 660, 686, 722, 742, 756, 784, 834, 870, 904, 909, 932, 956, 976, 1013, 1057, 1096, 1142, 1168, 1169, 1170, 1171, 1223, 1224, 1225, 1277, 1298, 1313, 1328, 1345, 1352, 1353, 1354, 1384, 1385, 1386, 1432, 1457, 1482, 1490, 1547, 1592, 1615, 1642, 1664, 1675, 1693, 1739, 1783, 1810, 1828, 1852, 1863, 1891, 1947, 1972, 2003, 2023, 2063, 2099, 2128, 2161, 2221, 2254, 2279, 2302, 2333, 2400, 2401, 2414, 2458, 2487, 2502, 2527, 2563, 2573, 2597, 2611, 2633, 2645, 2690, 2719, 2732, 2762, 2780, 2809, 2838, 2893, 2932, 2959, 2989, 3052, 3086, 3116, 3136, 3165, 3248, 3262, 3292, 3321, 3356, 3376, 3403, 3427, 3457, 3479, 3513, 3515, 3531, 3566, 3591, 3647, 3661, 3673, 3703, 3704, 3705, 3734, 3760, 3761, 3762, 3780, 3793, 3794, 3813, 3814, 3815, 3816, 3817, 3848, 3849, 3850, 3851, 3864, 3865, 3909, 3923, 3974, 4022, 4054, 4072, 4094, 4130, 4160, 4206, 4237, 4275, 4308, 4330, 4340, 